# Практическая работа №5 по вычислительной математике


Решить задачу Дирихле для уравнения Пуассона методом установления с упорядоченным набором оптимальных чебышевских параметров и точностью $\left|\dfrac{\Delta u}{u}\right|_1\le 0.001$

$$
\left\{
\begin{aligned}
&\frac{d^2u}{dx^2} + \frac{d^2u}{dy^2} = -2[x(1-x)+y(1-y)];\qquad 0\le x,y\le 1\\
&u|_{\Gamma} = 0
\end{aligned}
\right.
$$

In [1]:
import numpy as np
from math import sin, cos, pi

In [2]:
def cheb_order(N):
    if N == 1:
        return [1]
    iter_list = []
    while N > 0:
        iter_list.append(N)
        if (N % 2 == 0):
            N //= 2
        else:
            N -= 1
    overlined = [False for i in range(len(iter_list))]
    if iter_list[0] % 2 == 1 and iter_list[1] % 2 == 0:
        overlined[1] = True
    for i in range(1, len(overlined)-1):
        if iter_list[i] % 2 == 0 and iter_list[i-1] % 2 == 1 and iter_list[i+1] % 2 == 1:
            overlined[i] = True
    theta_old = []
    theta = [1]
    while len(iter_list) > 1:
        theta, theta_old = theta_old, theta
        m1 = iter_list.pop()
        m2 = iter_list[-1]
        theta = [0 for i in range(m2)]
        # переход от m к 2m без надчёркивания
        if m2 == 2*m1 and not overlined[-2]:
            for i in range(1, m1+1):
                theta[2*i-2] = theta_old[i-1]
                theta[2*i-1] = 4*m1 - theta[2*i-2]
        # переход от 2m без надчёркивания к 2m+1
        elif m2 == m1+1 and not overlined[-1]:
            for i in range(1, m1+1):
                theta[i-1] = theta_old[i-1]
            theta[m2-1] = m2*2 - 1
        # переход от m к 2m с надчёркиванием
        elif m2 == 2*m1 and overlined[-2]:
            for i in range(1, m1 + 1):
                theta[2*i-2] = theta_old[i-1]
                theta[2*i-1] = 4*m1 + 2 - theta[2*i-2]
        # переход от 2m c надчёркиванием к 2m+1
        elif m2 == m1+1 and overlined[-1]:
            for i in range(1, m1+1):
                theta[i-1] = theta_old[i-1]
            theta[m2-1] = m2
        overlined.pop()    
    return theta

In [3]:
def u_notstat(K, L, N):
    hx = 1./K
    hy = 1./L
    cheb = cheb_order(N)
    tau = [0 for i in range(N)]
    mx = 4*(K*K + L*L)
    mn = 2*pi*pi
    for n in range(N):
        tau[n] = 2 / ((mx + mn) + (mx - mn)*cos(pi * cheb[n]/2/N))
    u = np.zeros((K+1, L+1)) # начальные условия
    u1 = np.zeros((K+1, L+1)) # по бокам везде ноль
    
    for n in range(N):
        u, u1 = u1, u
        for k in range(1, K):
            for l in range(1, L):
                u[k][l] = u1[k][l] + tau[n] * ((u1[k+1][l] - 2*u1[k][l] + u1[k-1][l])/hx/hx + 
                                              (u1[k][l+1] - 2*u1[k][l] + u1[k][l-1])/hy/hy - func(k/K, l/L))
    return u

In [4]:
def func(x, y):
    return -2*(x*(1-x) + y*(1-y))

def an_solve(x, y):
    return x*(1-x)*y*(1-y)

In [20]:
K = 10
L = 10
N = 128
        
u_num = u_notstat(K, L, N)

u_an = np.zeros((11, 11))
for k in range(11):
    for l in range(11):
        u_an[k][l] = an_solve(k/10, l/10)

u_num2 = np.zeros((11, 11))
for k in range(11):
    for l in range(11):
        u_num2[k][l] = u_num[k * (K//10)][l * (L//10)]

delta_u = u_an - u_num2
        
delta = 0
rel_delta = 0

for k in range(11):
    for l in range(11):
        delta = max(delta, abs(delta_u[k][l]))
        if (u_an[k][l] != 0):
            rel_delta += abs(delta_u[k][l] / u_an[k][l])

print("delta = " + str(delta))
print("rel_delta = " + str(rel_delta))


delta = 2.7755575615628914e-17
rel_delta = 1.6852999956528007e-14


In [16]:
K = 10
L = 10
N = 32
        
u_an = np.zeros((11, 11))
for k in range(11):
    for l in range(11):
        u_an[k][l] = an_solve(k/10, l/10)

rel_delta = 1
while rel_delta >= 0.0001:
    N += 10
    u_num = u_notstat(K, L, N)

    u_num2 = np.zeros((11, 11))
    for k in range(11):
        for l in range(11):
            u_num2[k][l] = u_num[k * (K//10)][l * (L//10)]

    delta_u = u_an - u_num2

    delta = 0
    rel_delta = 0

    for k in range(11):
        for l in range(11):
            delta = max(delta, abs(delta_u[k][l]))
            if (u_an[k][l] != 0):
                rel_delta += abs(delta_u[k][l] / u_an[k][l])

print("N = " + str(N))
print("delta = " + str(delta))
print("rel_delta = " + str(rel_delta))


N = 50
delta = 2.5641922467289913e-08
rel_delta = 2.2920300764106197e-05


In [6]:
u_num2[7][7]

0.04409999160277521

In [7]:
u_an[7][7]

0.04410000000000001